# Exploring Geotaste codebase

In [1]:
import sys; sys.path.insert(0,'..')
from geotaste import *
import networkx as nx
# !pip install -qU inheritance_explorer
from inheritance_explorer import ClassGraphTree
def show_img(fn):
    from PIL import Image
    from IPython.display import display
    return display(Image.open(fn))
def show_inheritance(class_):
    cgt = ClassGraphTree(class_)
    fig=cgt.graph(rankdir='BT')
    imagefn=f'../docs/_static/{class_.__name__}.jpg'
    fig.write_jpg(imagefn)
    show_img(imagefn)
    return imagefn

In [2]:
def getcolor(x):
    if 'Figure' in x: return '#83bef9'
    if 'Dataset' in x: return '#f6e2be'
    return '#bda7ce'
    # if 'Panel' in x: return '#bda7ce'
    # elif 'Card' in x: return '#7e67da'
    # elif 'FigureFactory' in x: return '#3de1bb'
    # elif 'Figure' in x: return '#83bef9'
    # elif 'Dataset' in x: return '#a8f4f7'
    # return '#bda7ce'

In [3]:
# Code survey
# Dataset??
def get_graph(class_, fillcolor='white'):
    cgt = ClassGraphTree(class_)
    g = nx.nx_pydot.from_pydot(cgt.graph())
    id2label = {n:d['label'] for n,d in g.nodes(data=True)}
    og = nx.DiGraph()
    for n,d in g.nodes(data=True): 
        og.add_node(id2label[n], fillcolor=getcolor(id2label[n]), style='filled', **d)
    for a,b,d in g.edges(data=True): 
        og.add_edge(id2label[a], id2label[b], edge_type='is', **d)
    return og

In [4]:
dataset_g=get_graph(Dataset)
ff_g = get_graph(FigureFactory)
comp_g = get_graph(BaseComponent)

In [5]:
G = nx.compose_all([dataset_g, ff_g, comp_g])
G

In [6]:
for n in ff_g.nodes():
    obj = globals()[n]
    dset_obj = obj.dataset_class
    dset_name = dset_obj.__name__+'Dataset'
    if 'Factory' in n:
        print(n, dset_name)
        G.add_edge(n, dset_name, edge_type='uses')

FigureFactory CombinedDataset
LandmarksFigureFactory LandmarksDataset
CombinedFigureFactory CombinedDataset
ComparisonFigureFactory CombinedDataset


In [7]:
for n in comp_g.nodes():
    obj = globals()[n]
    try:
        ff_obj = obj.figure_factory
    except AttributeError:
        continue
    if ff_obj is None: continue
    
    ff_name = ff_obj.__name__
    print(n, ff_name)
    G.add_edge(n, ff_name, edge_type='uses')

MemberDOBCard MemberDOBFigure
MembershipYearCard MembershipYearFigure
BookYearCard BookYearFigure
EventYearCard EventYearFigure
EventMonthCard EventMonthFigure
MemberGenderCard MemberGenderFigure
MemberNationalityCard MemberNationalityFigure
MemberArrondCard MemberArrondFigure
AuthorGenderCard AuthorGenderFigure
BookGenreCard BookGenreFigure
AuthorNationalityCard AuthorNationalityFigure
EventTypeCard EventTypeFigure
MemberNameCard MemberNameFigure
BookTitleCard BookTitleFigure
AuthorNameCard AuthorNameFigure
MemberPanel CombinedFigureFactory
BookPanel CombinedFigureFactory
ComparisonPanel ComparisonFigureFactory


In [8]:
import networkx as nx
G.nodes(data=True)

NodeDataView({'Dataset': {'fillcolor': '#f6e2be', 'style': 'filled', 'label': 'Dataset', 'color': '#000000'}, 'LandmarksDataset': {'fillcolor': '#f6e2be', 'style': 'filled', 'label': 'LandmarksDataset', 'color': '#000000'}, 'MembersDataset': {'fillcolor': '#f6e2be', 'style': 'filled', 'label': 'MembersDataset', 'color': '#000000'}, 'MiniMembersDataset': {'fillcolor': '#f6e2be', 'style': 'filled', 'label': 'MiniMembersDataset', 'color': '#000000'}, 'DwellingsDataset': {'fillcolor': '#f6e2be', 'style': 'filled', 'label': 'DwellingsDataset', 'color': '#000000'}, 'MiniDwellingsDataset': {'fillcolor': '#f6e2be', 'style': 'filled', 'label': 'MiniDwellingsDataset', 'color': '#000000'}, 'BooksDataset': {'fillcolor': '#f6e2be', 'style': 'filled', 'label': 'BooksDataset', 'color': '#000000'}, 'MiniBooksDataset': {'fillcolor': '#f6e2be', 'style': 'filled', 'label': 'MiniBooksDataset', 'color': '#000000'}, 'CreatorsDataset': {'fillcolor': '#f6e2be', 'style': 'filled', 'label': 'CreatorsDataset', '

In [9]:
list(G.edges(data=True))


[('LandmarksDataset', 'Dataset', {'edge_type': 'is'}),
 ('MembersDataset', 'Dataset', {'edge_type': 'is'}),
 ('MiniMembersDataset', 'Dataset', {'edge_type': 'is'}),
 ('DwellingsDataset', 'Dataset', {'edge_type': 'is'}),
 ('MiniDwellingsDataset', 'Dataset', {'edge_type': 'is'}),
 ('BooksDataset', 'Dataset', {'edge_type': 'is'}),
 ('MiniBooksDataset', 'Dataset', {'edge_type': 'is'}),
 ('CreatorsDataset', 'Dataset', {'edge_type': 'is'}),
 ('EventsDataset', 'Dataset', {'edge_type': 'is'}),
 ('MiniEventsDataset', 'Dataset', {'edge_type': 'is'}),
 ('CombinedDataset', 'Dataset', {'edge_type': 'is'}),
 ('FigureFactory', 'CombinedDataset', {'edge_type': 'uses'}),
 ('MemberFigure', 'FigureFactory', {'edge_type': 'is'}),
 ('MemberDOBFigure', 'MemberFigure', {'edge_type': 'is'}),
 ('MembershipYearFigure', 'MemberFigure', {'edge_type': 'is'}),
 ('MemberGenderFigure', 'MemberFigure', {'edge_type': 'is'}),
 ('MemberNationalityFigure', 'MemberFigure', {'edge_type': 'is'}),
 ('MemberNationalityFigure',

In [10]:
panels = [globals()[x]() for x in globals() if 'Panel' in x]
panels

2023-09-25 13:57:53.692 | get_content | [CombinedPanel] component getting content | components.py:81
2023-09-25 13:57:53.693 | get_content | [Filter_1] component getting content | components.py:81
2023-09-25 13:57:53.694 | get_content | [Filter_2] component getting content | components.py:81
2023-09-25 13:57:53.694 | get_content | [Filter_1] component getting content | components.py:81
2023-09-25 13:57:53.695 | get_content | [Filter_1] component getting content | components.py:81
2023-09-25 13:57:53.695 | get_content | [MP-Filter_1] component getting content | components.py:81
2023-09-25 13:57:53.696 | content | [MemberNameCard-MP-Filter_1] getting content for input card | components.py:624
2023-09-25 13:57:53.696 | Combined | Combined() | datasets.py:585
2023-09-25 13:57:55.005 | get_content | [BP-Filter_1] component getting content | components.py:81
2023-09-25 13:57:55.006 | content | [BookTitleCard-BP-Filter_1] getting content for input card | components.py:624
2023-09-25 13:57:55.

In [ ]:
for panel in panels:
    for sub in panel.subcomponents:
        print(panel.__class__.__name__, sub.__class__.__name__)
        G.add_edge(panel.__class__.__name__, sub.__class__.__name__, edge_type='has')

MemberPanel MemberNameCard
MemberPanel MembershipYearCard
MemberPanel MemberDOBCard
MemberPanel MemberGenderCard
MemberPanel MemberNationalityCard
MemberPanel MemberArrondCard
BookPanel BookTitleCard
BookPanel BookYearCard
BookPanel BookGenreCard
BookPanel AuthorNameCard
BookPanel AuthorGenderCard
BookPanel AuthorNationalityCard
BookPanel EventYearCard
BookPanel EventMonthCard
CombinedPanel MemberPanel
CombinedPanel BookPanel
LeftPanel MemberPanel
LeftPanel BookPanel
RightPanel MemberPanel
RightPanel BookPanel
ComparisonPanel LeftPanel
ComparisonPanel RightPanel


In [ ]:
for a,b,d in G.edges(data=True): G.edges[a,b]['label'] = d['edge_type']
graph = nx.nx_pydot.to_pydot(G)
graph

In [ ]:
graph.set_rankdir('RL')
graph.write_png('graph.png')
!open graph.png

In [ ]:
import inspect
len(inspect.getsource(panels[0].__class__))

3643